# How to Build an AI Agent from Scratch in Raw Python

Self-link: https://colab.research.google.com/drive/1g31aYu3ylEurdQjAJtnSxGFcx9aSZa_D

Tutorial by Shrinivasan Sankar: https://www.ai-bites.net/lets-build-an-ai-agent-from-scratch-in-raw-python/?ref=ai-bites-newsletter

GitHub scource code: https://github.com/ai-bites/langgraph-crash-course/blob/main/0_simple_python_agent.ipynb?ref=ai-bites.net

Video: https://youtu.be/_TzW6F1NVsc

## Install necessary libraries

In [1]:
!pip install openai httpx dotenv

## Import necessary packages

In [2]:
from openai import OpenAI
import re
import httpx
import requests
import xml.etree.ElementTree as ET
import json

# Print the full version information string.
import sys
print(sys.version)

# Show the Python version.
import platform
print(f"Python version: {platform.python_version()}")

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Python version: 3.11.13


In [3]:
import os
import openai

from dotenv import load_dotenv


MODEL = "gpt-4o-mini"  # Chat-GPT model name

# Set your OPENAI_API_KEY in a "my_openai_key.env" file and load it.
load_dotenv("my_openai_key_2.env")  # OPENAI_API_KEY
# !echo $OPENAI_API_KEY

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'],)

# Check to see if the LLM works.
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "Hello there!"}],
)

# The anser should be: "Hello! How can I assist you today?"
print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


## Chatbot
Create a chatbot that takes raw messages, stacks them together, and uses the LLM to respond to the messages.

In [4]:
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})


    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.run_llm()
        self.messages.append({"role": "assistant", "content": result})
        return result


    def run_llm(self):
        completion = client.chat.completions.create(
            model=MODEL,
            temperature=0,
            messages=self.messages,
        )
        return completion.choices[0].message.content

## Agent
Wrap the above chatbot in Agent class to create ReAct Agent.

In [5]:
# abot = Agent(prompt)
action_re = re.compile('^Action: (\w+): (.*)$')


class Agent:
    def __init__(self, system_prompt="", max_turns=1, known_actions=None):
        self.max_turns = max_turns
        self.bot = ChatBot(system_prompt)
        self.known_actions = known_actions


    def run(self, question):
        i = 0  # iteration
        next_prompt = question

        while i < self.max_turns:  # loop through iterations
            i += 1
            result = self.bot(next_prompt)
            print(f"\n Iteration: {i} \n\n"
                  f"{result}\n\n")
            actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

            if actions:  # there is an action to run
                action, action_input = actions[0].groups()
                if action not in self.known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f" -- running {action} {action_input} --")

                # Make a request to Wikipedia, ArXiv or calculator using action functions.
                # print(self.known_actions)
                observation = self.known_actions[action](action_input)

                next_prompt = f"Observation: {observation}"
                print("Next prompt:", next_prompt)
            else:
                return

## Actions
Define actions as functions.

The arxiv_search function will invoke arxiv API to fetch data for the agent.

The Wikipedia function will use httpx to query Wikipedia and fetch knowledge about any topic unknown to the agent.

The calculate function is a Python hack to evaluate any equations
given to it.

In [6]:
def wikipedia(q):
    try:
        return httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": q,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]

    except IndexError:  # try Russian instead
        return httpx.get("https://ru.wikipedia.org/w/api.php", params={
            "action": "query",
            "list": "search",
            "srsearch": q,
            "format": "json"
        }).json()["query"]["search"][0]["snippet"]


def arxiv_search(q):
    ARXIV_NAMESPACE = '{http://www.w3.org/2005/Atom}'

    # url = f'http://export.arxiv.org/api/query?search_query=all:{q}&start=0&max_results=1'
    url = f'http://export.arxiv.org/api/query?search_query=all:{q}&start=0&max_results=3'
    res = requests.get(url)
    et_root = ET.fromstring(res.content)

    for entry in et_root.findall(f"{ARXIV_NAMESPACE}entry"):
        title = entry.find(f"{ARXIV_NAMESPACE}title").text.strip()
        summary = entry.find(f"{ARXIV_NAMESPACE}summary").text.strip()
    return json.dumps({"title" : title, "summary" : summary})


def calculate(what):
    return eval(what)

## Prompt
Prompting is quite crucial for ReAct agents. They generally work by few-shot prompting. Meaning, we need to provide a few examples of what is expected of the agent in the prompt itself. For this reason, we need to write a detailed prompt with a few examples. Below is the prompt for the task.

Nothing fancy here other than pro-level prompting. We have listed all the actions, explicitly mentioned, “run in a loop of Thought, Action, PAUSE, Observation” so that the agent can invoke actions and think of the perceived observations.

In [7]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia:
Returns a summary from searching Wikipedia

arxiv_search:
e.g. arxiv_search:
Returns a summary of research papers

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

## Test the Agent
It's time to test out our ReAct agent. Let's collate all the actions in a dictionary, create an instance of the agent, and invoke its run method to run the agent.

In [8]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "arxiv_search": arxiv_search
}
agent = Agent(prompt, max_turns=3, known_actions=known_actions)


def run_tests(question):
    bot = ChatBot(question)  # run chatbot without agent
    print(bot.run_llm())
    print(f"\n----------")
    agent.run(question)  # run agent

### 1st Question -- Easy

In [9]:
question = "What is the capital of Indonesia?"
run_tests(question)

The capital of Indonesia is Jakarta.

----------

 Iteration: 1 

Thought: I should look up Indonesia on Wikipedia to find out its capital.  
Action: wikipedia: Indonesia  
PAUSE


 -- running wikipedia Indonesia   --
Next prompt: Observation: <span class="searchmatch">Indonesia</span>, officially the Republic of <span class="searchmatch">Indonesia</span>, is a country in Southeast Asia and Oceania, between the Indian and Pacific oceans. Comprising over 17

 Iteration: 2 

Answer: The capital of Indonesia is Jakarta.




### 2nd Question -- Easy

In [10]:
question = "what is the sum of 7 and 5?"
run_tests(question)

The sum of 7 and 5 is 12.

----------

 Iteration: 1 

Thought: I can calculate the sum of 7 and 5 directly.  
Action: calculate: 7 + 5  
PAUSE


 -- running calculate 7 + 5   --
Next prompt: Observation: 12

 Iteration: 2 

Answer: The sum of 7 and 5 is 12.




### 3rd Question -- Difficult

In [11]:
question = "explain the lightrag paper"
run_tests(question)

The LightRAG paper presents a novel approach to enhancing the performance of retrieval-augmented generation (RAG) models, which combine retrieval mechanisms with generative models for tasks such as question answering and information retrieval. The key innovation in LightRAG is the introduction of a lightweight and efficient method for integrating retrieved documents into the generative process, aiming to improve both the quality of generated responses and the efficiency of the model.

### Key Components of LightRAG:

1. **Retrieval-Augmented Generation (RAG)**: RAG models typically retrieve relevant documents from a large corpus and use them to inform the generation of responses. This process can be computationally intensive, especially when dealing with large datasets.

2. **Lightweight Integration**: LightRAG proposes a more efficient way to incorporate retrieved information into the generative model. This involves optimizing the way documents are processed and reducing the computati

### 4th Qestion -- Difficult

In [12]:
question = "What is AI Bites?"
run_tests(question)

AI Bites is a term that can refer to short, digestible pieces of information or content related to artificial intelligence. This could include bite-sized articles, videos, podcasts, or social media posts that provide insights, updates, or explanations about AI concepts, technologies, trends, and applications. The goal of AI Bites is to make complex AI topics more accessible and understandable to a broader audience, allowing people to stay informed about developments in the field without needing extensive background knowledge. If you have a specific context or platform in mind for "AI Bites," please let me know!

----------

 Iteration: 1 

Thought: I should look up "AI Bites" on Wikipedia to find relevant information about it.  
Action: wikipedia: AI Bites  
PAUSE


 -- running wikipedia AI Bites   --
Next prompt: Observation: &quot;Retrieval Augmented Generation(RAG) — A quick and comprehensive introduction&quot;. <span class="searchmatch">ai</span>-<span class="searchmatch">bites</sp

### 5th Question -- Difficult

In [13]:
# Default answer is wrong. The answer after Wikipedia is write.
question = "Who is Alexander Tuzikov?"
run_tests(question)

As of my last knowledge update in October 2023, there is no widely recognized public figure or notable individual named Alexander Tuzikov. It's possible that he may be a private individual or a person who has gained prominence after that date. If you have specific context or details about him, I may be able to provide more relevant information.

----------

 Iteration: 1 

Thought: I should search for information about Alexander Tuzikov on Wikipedia to find relevant details about him.  
Action: wikipedia: Alexander Tuzikov  
PAUSE


 -- running wikipedia Alexander Tuzikov   --
Next prompt: Observation: Ryabova (Deputy Minister of Communications and Informatization), <span class="searchmatch">Alexander</span> <span class="searchmatch">Tuzikov</span> [ru] (General Director of the United Institute of Informatics Problems

 Iteration: 2 

Answer: Alexander Tuzikov is the General Director of the United Institute of Informatics Problems.




In [14]:
# Здесь ответ по умолчанию неверный, но после уточнения в Википедии -- верный.
question = "Кто такой Александр Васильевич Тузиков?"
run_tests(question)

Александр Васильевич Тузиков — это российский ученый, специалист в области физики и материаловедения. Однако, на момент моего последнего обновления информации в октябре 2023 года, нет широко известных данных о его деятельности или достижениях. Возможно, он является менее известной фигурой или работает в узкой области, не получившей широкой огласки. Если у вас есть конкретные вопросы о его работе или вкладе, пожалуйста, уточните, и я постараюсь помочь!

----------

 Iteration: 1 

Thought: Я должен поискать информацию о Александре Васильевиче Тузикове в Википедии, чтобы узнать о нем больше.  
Action: wikipedia: Александр Васильевич Тузиков  
PAUSE


 -- running wikipedia Александр Васильевич Тузиков   --
Next prompt: Observation: <span class="searchmatch">Александр</span> <span class="searchmatch">Васильевич</span> <span class="searchmatch">Тузиков</span> (род. 5 сентября 1958 года, Полоцк) — белорусский учёный, c 2008 по 2022 год -генеральный директор Объединенного института

 Iteratio

In [15]:
# Wrong answer
question = "What are the scientific works of Alexandra Kosareva?"
run_tests(question)

As of my last update in October 2023, I do not have specific information on the scientific works of Alexandra Kosareva. To find her publications or contributions to scientific research, I recommend checking academic databases such as Google Scholar, PubMed, or ResearchGate. You can also look for her work in university repositories or specific journals related to her field of study. If you have more context about her area of research, I might be able to provide more targeted advice on where to look.

----------

 Iteration: 1 

Thought: I should search for scientific works related to Alexandra Kosareva to provide relevant information.  
Action: arxiv_search: Alexandra Kosareva  
PAUSE


 -- running arxiv_search Alexandra Kosareva   --
Next prompt: Observation: {"title": "On some properties of Hopf manifolds", "summary": "We review old and new properties of Hopf manifolds from the point of view of\ntheir analytic and metric structure."}

 Iteration: 2 

Answer: One of the scientific work

In [16]:
# Wrong answer
question = "What are the scientific works of Dzmitry Paulenka?"
run_tests(question)

As of my last update in October 2023, I do not have specific information on the scientific works of Dzmitry Paulenka. To find his publications or contributions to scientific research, I recommend checking academic databases such as Google Scholar, ResearchGate, or institutional repositories. You can also look for his work in relevant journals or conference proceedings in his field of expertise. If you have access to a university library, they may also provide resources to help you find his research.

----------

 Iteration: 1 

Thought: I should search for scientific works related to Dzmitry Paulenka to gather relevant information.  
Action: arxiv_search: Dzmitry Paulenka  
PAUSE


 -- running arxiv_search Dzmitry Paulenka   --
Next prompt: Observation: {"title": "Homeomorphisms between limbs of the Mandelbrot set", "summary": "We prove that for every hyperbolic component of the Mandelbrot set, any two\nlimbs with equal denominators are homeomorphic so that the homeomorphism\npreserves

In [17]:
# Здесь ответ по умолчанию более подробный и полный, чем после поиска в Википедии.
question = "Что такое эффект зловещей долины?"
run_tests(question)

Эффект зловещей долины (или "uncanny valley" на английском) — это концепция, описывающая реакцию людей на роботов, анимацию или другие искусственные объекты, которые выглядят почти, но не совсем, как реальные люди. Когда внешний вид объекта становится всё более похожим на человеческий, наша реакция к нему становится положительной, но только до определённого момента. Если объект становится слишком похожим на человека, но всё же имеет некоторые отличия (например, неестественные движения, выражения лиц или детали), это может вызывать у людей чувство дискомфорта или даже отвращения.

Термин был введён японским робототехником Масахиро Мори в 1970-х годах. Эффект зловещей долины имеет важные последствия в области дизайна роботов, анимации, видеоигр и виртуальной реальности, поскольку создатели стремятся создать более естественные и привлекательные образы, избегая при этом "долины", где возникает негативная реакция.

----------

 Iteration: 1 

Thought: Я должен поискать информацию о "эффекте